In [1]:
import ipywidgets as widgets
from IPython.display import display

slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Slider:')
display(slider)


IntSlider(value=50, description='Slider:')

In [2]:
button = widgets.Button(description='Click me')
display(button)

def on_button_click(b):
    print("Button clicked!")

button.on_click(on_button_click)


Button(description='Click me', style=ButtonStyle())

Button clicked!
Button clicked!
